<a href="https://colab.research.google.com/github/21goldy/DevFest2.O/blob/main/DDOS_Project_Exhibition_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#@title Detection & Prevention of the Slowloris Attack utilizing the IsolationForest Algorithm & HTTP Port Blocking - [Group 16] { display-mode: "form" }


In [2]:
# importing the required modules of python 

import io
import pandas as pd
from google.colab import files

In [3]:
upload_file = files.upload() # uploading the collected CSV from wireshark for DOS analysis and attack detection 

Saving ddos_test1.csv to ddos_test1.csv


In [4]:
for fn in upload_file.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(upload_file[fn])))

User uploaded file "ddos_test1.csv" with length 3670085 bytes


In [5]:
data = pd.read_csv(io.StringIO(upload_file['ddos_test1.csv'].decode('utf-8'))) # reading the CSV file and storing it under the name "data" 

In [6]:
data.shape # defining the total rows and columns present (Here total rows can also be understood as the number of transmissions captured)

(28753, 7)

In [7]:
transmission_count = data.shape[0] # storing the rows/transmissions captured as "transmission_count"

In [8]:
data.head() # representing the starting 5 data sets in "data"

,No.,Time,Source,Destination,Protocol,Length,Info
0,1,0.000000,192.185.184.157,10.0.2.15,TCP,60,"80 > 45884 [FIN, ACK] Seq=1 Ack=1 Win=65535 ..."
1,2,0.040792,10.0.2.15,192.185.184.157,TCP,54,45884 > 80 [ACK] Seq=1 Ack=2 Win=63784 Len=0
2,3,0.079421,192.185.184.157,10.0.2.15,HTTP,570,HTTP/1.1 408 Request Timeout (text/html)
3,4,0.079437,10.0.2.15,192.185.184.157,TCP,54,45888 > 80 [ACK] Seq=1 Ack=517 Win=63784 Len=0
4,5,0.159054,192.185.184.157,10.0.2.15,TCP,60,"80 > 45994 [SYN, ACK] Seq=0 Ack=1 Win=65535 ..."


In [9]:
data.tail() # representing the ending 5 data sets in "data"

,No.,Time,Source,Destination,Protocol,Length,Info
28748,28749,856.829587,10.0.2.15,192.185.184.157,TCP,75,"49028 > 80 [PSH, ACK] Seq=1 Ack=1 Win=64240 ..."
28749,28750,856.829681,192.185.184.157,10.0.2.15,TCP,60,80 > 49028 [ACK] Seq=1 Ack=22 Win=65535 Len=0
28750,28751,856.829686,10.0.2.15,192.185.184.157,TCP,216,GET /?1198 HTTP/1.1 [TCP segment of a reassem...
28751,28752,856.829795,192.185.184.157,10.0.2.15,TCP,60,80 > 49028 [ACK] Seq=1 Ack=184 Win=65535 Len=0
28752,28753,856.829830,10.0.2.15,192.185.184.157,TCP,74,49030 > 80 [SYN] Seq=0 Win=64240 Len=0 MSS=1...


In [10]:
data.describe() # description of the data present in the dataset "data"

,No.,Time,Length
count,28753.000000,28753.000000,28753.000000
mean,14377.000000,387.491256,277.277849
std,8300.420481,243.610121,712.321905
min,1.000000,0.000000,54.000000
25%,7189.000000,187.627468,54.000000
50%,14377.000000,324.368047,60.000000
75%,21565.000000,610.634514,84.000000
max,28753.000000,856.829830,47150.000000


In [11]:
data.Source # listing the Source IP's present in the captured file

0        192.185.184.157
1              10.0.2.15
2        192.185.184.157
3              10.0.2.15
4        192.185.184.157
              ...       
28748          10.0.2.15
28749    192.185.184.157
28750          10.0.2.15
28751    192.185.184.157
28752          10.0.2.15
Name: Source, Length: 28753, dtype: object

In [12]:
source = data.Source 
source.nunique() # finding the unique IP's present in the Source column

33

In [13]:
destination = data.Destination
destination.nunique() # finding the unique IP's present in the Destination column

33

In [14]:
new_source = source.drop_duplicates() # removing the all the duplicates from the Source column
source_list = list(new_source) # creating list of all the unique IP's 
source_list
print(len(source_list))

33


In [15]:
major_ips = [] # empty list for keeping the majorly participating IP
minor_ips = [] # empty list for keeping the less participating IP
for i in source_list:
  analyse_ip = data[data.Source == i]
  transmission_of_ip = analyse_ip.shape[0]
  set_criteria = (transmission_count * 35)/100
  # print(set_criteria)
  if transmission_of_ip > set_criteria:
    major_ips.append(i)
  else:
    minor_ips.append(i)

In [16]:
major_ips # list of actively participating ip

['192.185.184.157', '10.0.2.15']

In [17]:
minor_ips # list of less active ip

['192.168.43.1',
 '13.227.214.62',
 '13.227.138.72',
 '142.250.195.74',
 '34.215.244.109',
 '142.250.183.227',
 '117.18.237.29',
 '142.250.192.68',
 '142.250.76.35',
 '142.250.182.131',
 '142.250.205.238',
 '142.250.77.97',
 '142.250.76.66',
 '142.250.182.66',
 '142.250.195.194',
 '23.15.34.57',
 '54.182.0.123',
 '172.217.163.202',
 '37.187.129.45',
 '13.235.26.255',
 '54.182.1.18',
 '157.240.16.20',
 '172.217.160.142',
 '142.251.12.156',
 '157.240.16.35',
 '13.227.166.81',
 '142.250.76.74',
 '142.250.195.78',
 '142.251.10.156',
 '142.250.192.3',
 '142.250.195.36']

In [18]:
#@title Analysis of the major IP's { display-mode: "form" }


In [19]:
for i in major_ips:
  dest_list_of_major_ip = list(data[data.Source == i].Destination.drop_duplicates()) # transmitted destination list of major IP
  major_df = pd.DataFrame(
      [
       (
           i,
           dest_list_of_major_ip,
           f'{data[data.Source == i].Protocol.max()}',
           f'{data[data.Source == i].Length.sum()}',
           'Attack'
       )
      ],
      columns=(
          'Source', 'Destination', 'Max_Protocol', 'Total_Length', 'Result'
          )
                 )
  major_df.to_csv (f'result_of_{i}.csv', index = False, header=True)
  print(major_df.head())
  print(major_df.Destination)

            Source  Destination Max_Protocol Total_Length  Result
0  192.185.184.157  [10.0.2.15]      TLSv1.3      5504122  Attack
0    [10.0.2.15]
Name: Destination, dtype: object
      Source  ...  Result
0  10.0.2.15  ...  Attack

[1 rows x 5 columns]
0    [192.185.184.157, 192.168.43.1, 13.227.214.62,...
Name: Destination, dtype: object


In [20]:
a = data[data.Source == '10.0.2.15'] 
b = a[a.Destination == '192.185.184.157']
b # transmission between the major IP "a" = '10.0.2.15' to "b" = '192.185.184.157'

b.to_csv (r'transmission_1.csv', index = False, header=True)

In [21]:
c = data[data.Source == '192.185.184.157']
d = c[c.Destination == '10.0.2.15']
d # transmission between the major IP "c" = '192.185.184.157' to "d" = '10.0.2.15'

d.to_csv (r'transmission_2.csv', index = False, header=True)

In [22]:
#@title Analysis of the Minor IP { display-mode: "form" }


In [23]:
for j in minor_ips:
  dest_list_of_minor_ip = list(data[data.Source == j].Destination.drop_duplicates()) # transmitted destination list of minor IP
  if data[data.Source == j].Length.sum() <= 13000:
    naive_df = pd.DataFrame(
    [
     (j, 
      dest_list_of_minor_ip,
      f'{data[data.Source == j].Protocol.max()}', 
      f'{data[data.Source == j].Length.sum()}', 
      'Naive'),
    ],
    columns=(
        'Source', 'Destination', 'Max_Protocol', 'Total_Length', 'Result'
        )
                 )
    # print(naive_df)
    # print('==========================================================')

  else:
    suspicious_df = pd.DataFrame(
    [
     (j, 
      dest_list_of_minor_ip,
      f'{data[data.Source == j].Protocol.max()}', 
      f'{data[data.Source == j].Length.sum()}', 
      'Suspicious'),
    ],
    columns=(
        'Source', 'Destination', 'Max_Protocol', 'Total_Length', 'Result'
        )
                 )
    # print(suspicious_df)
    # print('==========================================================')
  
